In [1]:
import pyopencl as cl
import pyopencl.characterize.performance as perf
import numpy as np
import time, math, sys

mf = cl.mem_flags

In [2]:
platforms = cl.get_platforms()
cq = []
for platform in platforms:
    for dev in platform.get_devices():
        context = cl.Context(devices=[dev])
        queue = cl.CommandQueue(context=context, properties=cl.command_queue_properties.PROFILING_ENABLE)
        cq.append(( context, queue ))

cq

[(<pyopencl.Context at 0x56354c227100 on <pyopencl.Device 'Tesla K80' on 'NVIDIA CUDA' at 0x56354bcb3040>>,
  <pyopencl.cffi_cl.CommandQueue at 0x7f0c301304d0>),
 (<pyopencl.Context at 0x56354cd25760 on <pyopencl.Device 'pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz' on 'Portable Computing Language' at 0x56354bdef9a0>>,
  <pyopencl.cffi_cl.CommandQueue at 0x7f0c30130550>)]

## create test harness and measure time

In [3]:
from IPython.display import display, HTML
def prettyprint(table):
    display(HTML('<table><tr>{0}</tr></table>'.format(
    "</tr><tr>".join( '<td>{}</td>'.format( 
        '</td><td>'.join(str(_) for _ in row)) for row in table )
    )))

In [4]:
def test(harness_builder = lambda context, queue, N: (lambda:None), count=10, N=1024):
    out = [ [ " " ] + [ cqu[0].devices[0].name for cqu in cq ], [ "Program Time (ms)"], [ "Copy buffer out time (ms)"], [ "MFLOPS" ] ]
    for context, queue in cq:
        pgmtimes = []
        copytimes = []
        harness, expected_reply = harness_builder(context, queue, N)
        print >> sys.stderr, "Testing with", context.devices[0].name
        result = None
        for i in xrange(count):
            try:
                result, evt_pgm, evt_copyout = harness()
            except Exception, e:
                print >> sys.stderr, "Exception on", context.devices[0].name, str(e)
                break
            pgmtimes.append((evt_pgm.profile.end-evt_pgm.profile.start)*1e-6)
            copytimes.append((evt_copyout.profile.end-evt_copyout.profile.start)*1e-6)
            if expected_reply is not None:
                if ( (result-expected_reply) > 0.01 ).any():
                    print >> sys.stderr, "Warning! wrong result on", context.devices[0].name
            else:
                print >> sys.stderr, 'No expected reply'
        if pgmtimes:
            out[1].append('{0:.3f}'.format( np.average(pgmtimes) ) )
            out[2].append('{0:.3f}'.format( np.average(copytimes) ) )
            out[3].append('{0:.0f}'.format( 2.0 * N * N * N/(1000.0*np.average(pgmtimes)) ) )
        else:
            out[1].append("N/A")
            out[2].append("N/A")
            out[3].append("N/A")

    prettyprint(out)
    

        

## view CPU/GPU capabilities

In [5]:
out = [ [ " " ] + [ cqu[0].devices[0].name for cqu in cq ] ]
out += [ [ "max work item size" ] + [ cqu[0].devices[0].max_work_item_sizes for cqu in cq ] ]
prettyprint(out)

,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
max work item size,"[1024L, 1024L, 64L]","[4096L, 4096L, 4096L]"


In [6]:
for ctx, queue in cq:
    prof_overhead, latency = perf.get_profiling_overhead(ctx)
    print("command latency: %g s" % latency)
    print("profiling overhead: %g s -> %.1f %%" % (
            prof_overhead, 100*prof_overhead/latency))
    queue = cl.CommandQueue(
            ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)

    print("empty kernel: %g s" % perf.get_empty_kernel_time(queue))
    print("float32 add: %g GOps/s" % (perf.get_add_rate(queue)/1e9))

    for tx_type in [
            perf.HostToDeviceTransfer,
            perf.DeviceToHostTransfer,
            perf.DeviceToDeviceTransfer]:
        print("----------------------------------------")
        print(tx_type.__name__)
        print("----------------------------------------")

        print("latency: %g s" % perf.transfer_latency(queue, tx_type))
        for i in range(6, 31, 2):
            bs = 1 << i
            try:
                result = "%g GB/s" % (perf.transfer_bandwidth(queue, tx_type, bs)/1e9)
            except Exception as e:
                result = "exception: %s" % e.__class__.__name__
            print("bandwidth @ %d bytes: %s" % (bs, result))


command latency: 3.53456e-05 s
profiling overhead: 1.01333e-05 s -> 28.7 %
empty kernel: 3.7255e-05 s
52 1879028.31766 0
104 3835450.78812 0
208 7988087.50095 0
416 15955790.065 0
832 31998633.0763 0
1664 64255221.6445 0
3328 130489866.212 0
6656 259592689.984 0
13312 522724522.54 0
26624 1050030939.06 0
53248 2083191403.69 0
106496 4166000294.69 0
212992 8363494486.62 0
425984 16325922387.3 0
851968 33097977431.5 0
1703936 61802616313.7 0
3407872 81737704351.0 0
6815744 93056488803.2 0
13631488 99716360113.0 0
27262976 1.03450869848e+11 0
float32 add: 51725.4 GOps/s
----------------------------------------
HostToDeviceTransfer
----------------------------------------
latency: 4.49203e-05 s
bandwidth @ 64 bytes: 0.00142473 GB/s
bandwidth @ 256 bytes: 0.00569969 GB/s
bandwidth @ 1024 bytes: 0.0226423 GB/s
bandwidth @ 4096 bytes: 0.089303 GB/s
bandwidth @ 16384 bytes: 0.333265 GB/s
bandwidth @ 65536 bytes: 1.08777 GB/s
bandwidth @ 262144 bytes: 2.80674 GB/s
bandwidth @ 1048576 bytes: 4.8

## Basic program

In [7]:
def harnessbuilder_basic(context, queue, N):
    h_A = np.random.rand(N,N).astype(np.float32)
    h_B = np.random.rand(N,N).astype(np.float32)
    h_C = np.empty([N,N]).astype(np.float32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource_basic = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j = get_global_id(1);
            __private uint k;
            __private float tmp = 0.0f;
            for (k=0; k<count; k++) {
                tmp += A[ i*count + k ] * B [ k*count + j ];
            }
            out[i*count + j] = tmp;
        }
    """
    program = cl.Program(context, kernelsource_basic).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        evt_pgm = mmul(queue, h_A.shape, None, d_A, d_B, d_C, N)
        evt_copy = cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, evt_pgm, evt_copy
    
    return run_mmul, expected_reply

In [8]:
test(harnessbuilder_basic, 10, 1024)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),224.517,1367.970
Copy buffer out time (ms),0.481,1.396
MFLOPS,9565,1570


## version with row as work-item

In [9]:
def harnessbuilder_row_as_workitem(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j, k;
            for (j=0; j<count; j++)
            {
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += A[ i*count + k ] * B [ k*count + j ];
                }
                out[i*count + j] = tmp;
            }
        }
    """
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        evt_pgm =mmul(queue, (count,), (32,), d_A, d_B, d_C, count)
        evt_copy =cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, evt_pgm, evt_copy
    
    return run_mmul, expected_reply

In [10]:
test(harnessbuilder_row_as_workitem, 10, 1024)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),474.540,1413.832
Copy buffer out time (ms),0.465,1.288
MFLOPS,4525,1519


## version with row stored in private memory

In [11]:
def harnessbuilder_row_cached_in_private(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint j, k;
            __private float Arwk["""+str(count)+"""];
            
            for (k=0; k<count; k++)
                Arwk[k] = A[i*count+k];
            
            for (j=0; j<count; j++)
            {
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += Arwk[k] * B [ k*count + j ];
                }
                out[i*count + j] = tmp;
            }
        }
    """
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,np.uint32])

    def run_mmul():
        evt_pgm = mmul(queue, (count,), None, d_A, d_B, d_C, count)
        evt_copy = cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, evt_pgm, evt_copy
    
    return run_mmul, expected_reply

In [12]:
test(harnessbuilder_row_cached_in_private, 10, 1024)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),223.477,2076.648
Copy buffer out time (ms),0.466,1.285
MFLOPS,9609,1034


## version with local memory for the column

In [13]:
def harnessbuilder_privateA_localB(context, queue, count):
    h_A = np.random.rand(count**2).astype(np.float32)
    h_B = np.random.rand(count**2).astype(np.float32)
    h_C = np.empty([count**2]).astype(np.float32)
    expected_reply = h_A.reshape(count,count).dot(h_B.reshape(count,count)).reshape(count**2)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes)
    kernelsource = """
       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global float* out,
                          __local float* Bwrk,
                          uint count)
        {
            __private uint i = get_global_id(0);
            __private uint iloc = get_local_id(0);
            __private uint nloc = get_local_size(0);
            __private uint j, k;
            __private float Arwk["""+str(count)+"""];
            
            for (k=0; k<count; k++)
                Arwk[k] = A[i*count+k];
            
            for (j=0; j < count; j++)
            {
                for( k = iloc; k<count; k+=nloc)
                {
                    Bwrk[k] = B[ k*count + j ];
                }
            
                barrier(CLK_LOCAL_MEM_FENCE);
                
                __private float tmp = 0.0f;
                for (k=0; k<count; k++) {
                    tmp += Arwk[k] * Bwrk[k];
                }
                out[i*count + j] = tmp;
                
                barrier(CLK_LOCAL_MEM_FENCE);
            }
                
        }
    """
    local = cl.LocalMemory(count*4)
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,np.uint32])

    def run_mmul():
        evt_pgm = mmul(queue, (count,), None, d_A, d_B, d_C, local, count)
        evt_copy = cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, evt_pgm, evt_copy
    
    return run_mmul, expected_reply

In [14]:
test(harnessbuilder_privateA_localB, 2, 1024)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),156.594,2581.983
Copy buffer out time (ms),0.517,1.297
MFLOPS,13714,832


## version split by blocks

In [15]:
def harnessbuilder_blocks(context, queue, count):
    h_A = np.random.rand(count,count).astype(np.float32)
    h_B = np.random.rand(count,count).astype(np.float32)
    h_C = np.empty([count,count]).astype(np.float32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes*4)
    
    BLOCK_SIZE = 32
    kernelsource = """

        #define BLOCK_SIZE """+str(BLOCK_SIZE)+"""
        //#define count """+str(count)+"""
        
        #define ASUB(i, j) Asub[i + j*BLOCK_SIZE]
        #define BSUB(i, j) Bsub[i + j*BLOCK_SIZE]



       __kernel void mmul(__global const float* A,
                          __global const float* B,
                          __global       float* out,
                          __local float* Asub,
                          __local float* Bsub,
                          __private uint count)
        {
        
            // this processing element has to compute out[ get_global_id(0), get_global_id(1) ]
            
            uint block_id_x = get_group_id(0);
            uint block_id_y = get_group_id(1);
            
            uint x_thread = get_local_id(0);
            uint y_thread = get_local_id(1);

            uint aStart = block_id_y * BLOCK_SIZE * count;
            uint aStep = BLOCK_SIZE;
            uint aEnd = aStart + count - 1;
            
            uint bStart = block_id_x * BLOCK_SIZE;
            uint bStep = BLOCK_SIZE * count;
            
            __private float Csub = 0.0f;
            
            for (uint a = aStart, b = bStart;
                 a <= aEnd;
                 a += aStep, b+= bStep)
                 {
                     ASUB(x_thread, y_thread) = A[a + count * y_thread + x_thread];
                     BSUB(x_thread, y_thread) = B[b + count * y_thread + x_thread];
                     
                     barrier(CLK_LOCAL_MEM_FENCE);
                     
                     #pragma unroll
                     for( uint k =0; k < BLOCK_SIZE; ++k)
                         Csub += ASUB(k, y_thread) * BSUB(x_thread, k);
                         
                     barrier(CLK_LOCAL_MEM_FENCE);
                 }
                 
            out[get_global_id(1) * count + get_global_id(0)] = Csub;
                
        }
    """
    local_a = cl.LocalMemory(4*(BLOCK_SIZE**2))
    local_b = cl.LocalMemory(4*(BLOCK_SIZE**2))
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,None,np.uint32])

    def run_mmul():
        event_pgm = mmul(queue, (count,count), (BLOCK_SIZE,BLOCK_SIZE), d_A, d_B, d_C, local_a, local_b, count)
        event_copy_out = cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, event_pgm, event_copy_out
    
    return run_mmul, expected_reply

In [16]:
test(harnessbuilder_blocks, 1, 1024*2)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),72.802,9168.330
Copy buffer out time (ms),1.842,5.059
MFLOPS,235981,1874


## version split by blocks - Integer!

In [17]:
def harnessbuilder_blocks_int(context, queue, count):
    h_A = np.random.rand(count,count).astype(np.int32)
    h_B = np.random.rand(count,count).astype(np.int32)
    h_C = np.empty([count,count]).astype(np.int32)
    expected_reply = h_A.dot(h_B)
    
    d_A = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_A)
    d_B = cl.Buffer(context, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=h_B)
    d_C = cl.Buffer(context, mf.WRITE_ONLY, h_C.nbytes*4)
    
    BLOCK_SIZE = 16
    kernelsource = """

        #define BLOCK_SIZE """+str(BLOCK_SIZE)+"""
        //#define count """+str(count)+"""
        
        #define ASUB(i, j) Asub[i + j*BLOCK_SIZE]
        #define BSUB(i, j) Bsub[i + j*BLOCK_SIZE]



       __kernel void mmul(__global const int* A,
                          __global const int* B,
                          __global       int* out,
                          __local int* Asub,
                          __local int* Bsub,
                          __private uint count)
        {
        
            // this processing element has to compute out[ get_global_id(0), get_global_id(1) ]
            
            uint block_id_x = get_group_id(0);
            uint block_id_y = get_group_id(1);
            
            uint x_thread = get_local_id(0);
            uint y_thread = get_local_id(1);

            uint aStart = block_id_y * BLOCK_SIZE * count;
            uint aStep = BLOCK_SIZE;
            uint aEnd = aStart + count - 1;
            
            uint bStart = block_id_x * BLOCK_SIZE;
            uint bStep = BLOCK_SIZE * count;
            
            __private int Csub = 0;
            
            for (uint a = aStart, b = bStart;
                 a <= aEnd;
                 a += aStep, b+= bStep)
                 {
                     ASUB(x_thread, y_thread) = A[a + count * y_thread + x_thread];
                     BSUB(x_thread, y_thread) = B[b + count * y_thread + x_thread];
                     
                     barrier(CLK_LOCAL_MEM_FENCE);
                     
                     #pragma unroll
                     for( uint k =0; k < BLOCK_SIZE; ++k)
                         Csub += ASUB(k, y_thread) * BSUB(x_thread, k);
                         
                     barrier(CLK_LOCAL_MEM_FENCE);
                 }
                 
            out[get_global_id(1) * count + get_global_id(0)] = Csub;
                
        }
    """
    local_a = cl.LocalMemory(4*(BLOCK_SIZE**2))
    local_b = cl.LocalMemory(4*(BLOCK_SIZE**2))
    program = cl.Program(context, kernelsource).build()
    mmul = program.mmul
    mmul.set_scalar_arg_dtypes([None,None,None,None,None,np.uint32])

    def run_mmul():
        evt_pgm = mmul(queue, (count,count), (BLOCK_SIZE,BLOCK_SIZE), d_A, d_B, d_C, local_a, local_b, count)
        evt_copy = cl.enqueue_copy(queue, h_C, d_C)
        queue.finish()
        return h_C, evt_pgm, evt_copy
    
    return run_mmul, expected_reply

In [18]:
test(harnessbuilder_blocks_int, 1, 1024*2)

Testing with Tesla K80
Testing with pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz


,Tesla K80,pthread-Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Program Time (ms),112.748,9939.520
Copy buffer out time (ms),2.207,4.934
MFLOPS,152374,1728
